# Convolutional Neural Network in Keras

Bulding a Convolutional Neural Network to classify Fashion-MNIST accessories.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import os
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Layer, Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, LeakyReLU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

from keras_contrib.layers.advanced_activations import SineReLU

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Load data

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 2s 0us/step


#### Preprocess data
Flatten and normalise input data.

In [4]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_test = X_test.astype("float32")/255.

In [5]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### Test Predictions

In [6]:
model_output = 'model_output/weights-cnn-fashion-mnist.hdf5'
saved_model = keras.models.load_model(model_output)

#### Test Final Accuracy

In [7]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 28s 3ms/step
Final loss: 0.2991, final accuracy: 0.8875


#### Log model and metrics in MLFlow

In [8]:
# If it doesn't show anything is because you forgot to copy the known_hosts. Please read the README again and make sure something sensible appears here.
!cat ~/.ssh/known_hosts

|1|uvV5+M7H1MUNPlwDr1H8CraFsB4=|9XhBYy77L95tdvjb2J8ZwerQ628= ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAICoUZuEqW0rbK+8f4mNpsGgeJYulGTHkYB3com6YWEE7
|1|Kw8J908J86S90tB/394KQY5+KR0=|zBVjyXKHiJ/iPNTiuLBEEroWt00= ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCslZ7pp0imLQ7HUFM3sU10xBbnk9CFBRtO/6ZO9IuJGGlTmd46zV3nUq3lvJKGVMq7GsfJbunsHa7acVTXr5GFPhZjcMK4N89Q/GxepwljO77GHwKKLctPAqNX1aEMgjoZUOAbzAJ+EFNbSHRIVhZE5iDa6wFcqyOYIx6TlGCzRb22xUcJbQ8Wkeehfz1X7Pw2HHSS8yrUF+zyCI8+7n0GVHj1T5cj6Ainh3626u4k4Fvo7kWScv/vFGYM86PQZHtZ/HDeWNO+vjnNrNNGbKL2UKPfcNLDz6Pp3ll1oxyz5Qc6VF7aH1tXO8xLaHclETXIMgKyBLp1TxeCbF3++OrA2TjPGK46y0yYdihmC6wGWhJyjXfUz+quLgH7rCyMXVSWSdvYWd1ePRPXjvGz9mwqZpSg7qW68+6fiOc1r+nByuPAUFxdS3lAKwdqQeA+FBOYto0mSC71f0golHXonaVpn6bBvxfHBGBK/14nbLPmscjgM6Z+6Pfrlpm0Cp7lwgY/cEsWwXiIMfdJ419UOgVgG9pWg8oKIP9H1acESWUnVt+6Hl/ExkZE1x0XDyoAm1mLpRRvDuA2UCCVDFHtzrJ0tZKdwa7ufDJK5mKVn8pzSvVYQ5PkNiQhdGQPczqyG6PWLMKeEyPj/f75LtRl9/wnKWlDtUH12fVwHeEXbWTcIQ==


In [9]:
import mlflow
import mlflow.keras

mlflow.tracking.set_tracking_uri('http://ekholabs-mlflow-server:5500')

with mlflow.start_run(experiment_id=1, source_version='0.0.1') as run:
    mlflow.log_param(key='backend', value='tensorflow')
    mlflow.log_param(key='backend_version', value='1.10.0')
    mlflow.log_metric('final_loss', final_loss)
    mlflow.log_metric('final_accuracy', final_acc)
    mlflow.log_artifact(model_output)
